In [ ]:
import sys, os
pardir = os.path.abspath(os.path.join(os.path.dirname('__file__'), os.path.pardir))
sys.path.append(pardir)

import numpy as np
import yaafelib
import matplotlib.pyplot as plt
from scipy.io import wavfile
from read_labels import read_labels

%matplotlib inline

In [ ]:
rootdir = '/home/tracek/Data/Birdman/'
filename = os.path.join(rootdir, 'raw/STHELENA-02_20140605_200000_1.wav')
path = os.path.join(root_path, filename) 
sheets = read_labels('/home/tracek/Data/Birdman/labels/sthelena_labels.xls')
filename_noext = os.path.splitext(os.path.basename(filename))[0]
sheet = sheets[filename_noext]

sample_start = 3 * 60
sample_len = 60 # seconds
sample_end = sample_start + sample_len
step_size = 512 # no. of samples

In [ ]:
rate, signal = wavfile.read(path)
sample = signal[sample_start * rate:(sample_start + sample_len) * rate]

feature_plan = yaafelib.FeaturePlan(sample_rate=rate, normalize=True)
features_config_path = os.path.join(pardir, 'features.config') 
success = feature_plan.loadFeaturePlan(features_config_path)
engine = yaafelib.Engine()
engine.load(feature_plan.getDataFlow())
feats = engine.processAudio(sample.reshape(1, -1).astype('float64'))

In [ ]:
condition = (sheet['Time Start'] > sample_start) & (sheet['Time End'] < sample_end)
sheet_sample = sheet_selected[condition]

In [ ]:
print('No. of calls: {}'.format(len(sheet_sample)))
sheet_sample

In [ ]:
def plot_feature(feature, start=0, sheet=None):
    cutoff = 5 # to counter edge effects when calculatig derivatives
    feature = feature[cutoff:-cutoff,:]
    x_spacing = np.linspace(sample_start, sample_start+sample_len, len(feature))

    no_figures = feature.shape[1]
    if no_figures == 1:
        plt.figure(figsize=(15,5))
        plt.plot(x_spacing, feature)
        if sheet is not None:
            plot_feature_location(plt, sheet)
    else:
        f, ax = plt.subplots(no_figures, 1, sharex=True, figsize=(15,15))
        for idx in range(no_figures):
            ax[idx].plot(x_spacing, feature[:, idx])
            if sheet is not None:
                plot_feature_location(ax[idx], sheet)

def plot_feature_location(axis, sheet):
    for index, row in sheet.iterrows():
        if row['Species'] == 'Storm Petrel':
            axis.axvspan(xmin=row['Time Start'], xmax=row['Time End'], color='red', alpha=0.1)
        else:
            axis.axvspan(xmin=row['Time Start'], xmax=row['Time End'], color='green', alpha=0.1)

In [ ]:
plot_feature(feats['LSF_d2'], start=sample_start, sheet=sheet_sample)

In [ ]:
sheet_sample